# 5.10 批量归一化(batch normalization)

关于BN：

+ BN能让较深的神经网络的训练变得更加容易。

+ 模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络的中间输出，通过这种调整，使得神经网络的中间输出值更加稳定。

+ 对全连接层和卷积层做批量归一化的方法有所不同：

**1.对全连接层做批量归一化**

+ 批量归一化层位于全连接层的仿射变换和激活函数之之间。

使用批量归一化的全连接层的输出为：

$$
\phi(BN(x))
$$

其中，x为批量归一化的输入，由仿射变换$x=Wu+b$得到。$\phi$为激活函数。

如果一个个小批量有m个样本:

$u^{(1)},u^{(2)},...,u^{(m)}$，

则经过放射变化，得到:

$x^{(1)},x^{(2)},...,x^{(m)}$

这些就是批量归一化的输入，对于任意$x^{(i)}$，有
$x^{(i)}\in R^d, 1\leq i \leq m$，

做批量归一化处理: 

$$
\hat{x}^{(i)}\leftarrow \frac{x^{(i)}-\mu}{\sqrt{(\sigma ^2 + \epsilon)}},

$$

其中，$\mu$是这一批量样本的均值，$\sigma$是这一批量样本的方差。计算如下：

$$
\mu \leftarrow=\frac{1}{m}\sum_{i=1}^{m}x^{(i)},\\
\sigma ^2 \leftarrow \frac{1}{m}\sum_{i=1}^{m}(x^{(i)}-\mu)^2,
$$

上述批量归一化，引入了两个可以学习的模型参数：拉伸参数$\gamma$和偏移参数$\beta$，这两个参数与$x^{(i)}$维数相同。

于是得到:

$y^{(i)}\leftarrow \gamma \odot (\hat{x}^{(i)}+\beta)$

其中，$\odot$表示元素乘法。

计算之后的$y_{i}$就是批量归一化的输出。

Note:

如果学习得到的$\gamma=\sqrt{(\delta ^2 +\epsilon)}$和$\beta=\mu$，则等价于没有做批量归一化。

**2.对卷积层做批量归一化**

+ 对于卷积层来说，批量归一化发生在卷积计算之后，激活函数之前，

+ 如果卷积计算输出多个通道，则需要对每个通道的输出分别做批量归一化，

+ 多通道时，每个通道都拥有独立的拉伸和偏移参数，

+ 单通道时，如果卷积计算输出的宽和高是p\*q，则需要对m\*p\*q个元素，同时做批量归一化。其中,m是小批量中样本的个数。

**预测时的批量归一化**

一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。

和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

自定义实现批量归一化层：

In [1]:
import time

import torch 
from torch import nn,optim 
import torch.nn.functional as F  

import sys
sys.path.append('..')
import d2l_pytorch as d2l 

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def batch_norm(is_training,X,gamma,beta,moving_mean,moving_var,eps,momentum):
    # 区分当前模式
    if not is_training:
        # 预测模式下，直接使用出入的移动平均，由此来计算得到的均值和方差
        X_hat=(X-moving_mean)/torch.sqrt(moving_var+eps)
    else:
        # 训练模型
        assert len(X.shape) in (2,4)
        if len(X.shape)==2:
            # 是全连接层，则计算特征维上的均值和方差
            mean=X.mean(dim=0)
            var=((X-mean)**2).mean(dim=0)
        else:
            # 是二维卷积层，则计算每个通道维上的均值和方差
            # 保持X的形状，因为之后需要用到广播计算
            mean=X.mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
            var=((X-mean)**2).mean(dim=0,keepdim=True).mean(dim=2,keepdim=True).mean(dim=3,keepdim=True)
        # 训练模式下，使用小批量的均值和方差
        X_hat=(X-mean)/torch.sqrt(var+eps)
        # 更新移动平均的均值和方差
        moving_mean=momentum*moving_mean + (1.0-momentum)*mean
        moving_var=momentum*moving_var + (1.0-momentum)*var 
    # 偏移和拉伸，得到BN输出
    Y=gamma*X_hat+beta 
    return Y,moving_mean,moving_var

自定义一个BatchNorm层，它保存参与求梯度和迭代的拉伸参数gamma和偏移参数beta。同时维护移动平均得到的均值和方差，用于模型预测。


In [3]:
class BatchNorm(nn.Module):
    def __init__(self,num_features,num_dims):
        # num_features为输入和输出的维度数,num_dims取值为2,4，用以判断是卷积层还是全连接层
        super(BatchNorm,self).__init__()
        if num_dims==2:
            shape=(1,num_features)
        else:
            shape=(1,num_features,1,1)
        #  # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma=nn.Parameter(torch.ones(shape))
        self.beta=nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean=torch.zeros(shape)
        self.moving_var=torch.zeros(shape)
    def forward(self,X):
        # 如果X不在内存，则将moving_mean和moving_var移动到显存上
        if self.moving_mean.device!=X.device:
            self.moving_mean=self.moving_mean.to(X.device)
            self.moving_var=self.moving_var.to(X.device)
            # 保存更新过的moving_mean和moving_var
        # Module实例的traning属性默认为true, 调用.eval()后设成false
        Y,self.moving_mean,self.moving_var=batch_norm(self.training,X,self.gamma,self.beta,self.moving_mean,self.moving_var,eps=1e-5,momentum=0.9)
        return Y

修改LeNet网络，添加批量归一化层

In [4]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        # 卷积块包含2层：卷积层，激活函数，最大池化，卷积层，激活函数，最大池化
        # in_channel=1,out_channle=6,kernelz-size=5
        self.conv=nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            BatchNorm(6,num_dims=4),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5),
            BatchNorm(16,num_dims=4),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.fc=nn.Sequential(
            d2l.FlattenLayer(),
            nn.Linear(in_features=16*4*4,out_features=120),
            BatchNorm(120,num_dims=2),
            nn.Sigmoid(),
            nn.Linear(in_features=120,out_features=84),
            BatchNorm(84,num_dims=2),
            nn.Sigmoid(),
            nn.Linear(in_features=84,out_features=10)
        )
    def forward(self,img):
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

In [5]:
net=LeNet()

In [6]:
# test
# 将输入的高和宽从224降到96，简化计算

X=torch.rand(1,1,28,28)

for blk in net.children():
    X=blk(X)
    print('out shape:',X.shape)

out shape: torch.Size([1, 16, 4, 4])
out shape: torch.Size([1, 10])


In [7]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

lr, num_epochs = 0.001,1   # 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [8]:
d2l.train_ch05(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)


training on  cpu
epoch 0/1, iter 0/234, loss 2.335
epoch 0/1, iter 1/234, loss 2.233
epoch 0/1, iter 2/234, loss 2.148
epoch 0/1, iter 3/234, loss 2.062
epoch 0/1, iter 4/234, loss 2.004
epoch 0/1, iter 5/234, loss 1.959
epoch 0/1, iter 6/234, loss 1.908
epoch 0/1, iter 7/234, loss 1.859
epoch 0/1, iter 8/234, loss 1.848
epoch 0/1, iter 9/234, loss 1.817
epoch 0/1, iter 10/234, loss 1.817
epoch 0/1, iter 11/234, loss 1.803
epoch 0/1, iter 12/234, loss 1.741
epoch 0/1, iter 13/234, loss 1.744
epoch 0/1, iter 14/234, loss 1.745
epoch 0/1, iter 15/234, loss 1.718
epoch 0/1, iter 16/234, loss 1.687
epoch 0/1, iter 17/234, loss 1.712
epoch 0/1, iter 18/234, loss 1.649
epoch 0/1, iter 19/234, loss 1.639
epoch 0/1, iter 20/234, loss 1.627
epoch 0/1, iter 21/234, loss 1.639
epoch 0/1, iter 22/234, loss 1.623
epoch 0/1, iter 23/234, loss 1.618
epoch 0/1, iter 24/234, loss 1.599
epoch 0/1, iter 25/234, loss 1.579
epoch 0/1, iter 26/234, loss 1.590
epoch 0/1, iter 27/234, loss 1.566
epoch 0/1, it

## BatchNorm简洁实现

使用nn.BatchNorm1d()和nn.BatchNorm2d()类，分别用于全连接层和卷积层的批量归一化。

In [7]:
class LeNet2(nn.Module):
    def __init__(self):
        super(LeNet2,self).__init__()
        # 卷积块包含2层：卷积层，激活函数，最大池化，卷积层，激活函数，最大池化
        # in_channel=1,out_channle=6,kernelz-size=5
        self.conv=nn.Sequential(
            nn.Conv2d(1,6, kernel_size=5),
            nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6,16,5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2)
        )
        self.fc=nn.Sequential(
            d2l.FlattenLayer(),
            nn.Linear(in_features=16*4*4,out_features=120),
            nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120,84),
            nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84,10)
        )
    def forward(self,img):
        feature=self.conv(img)
        output=self.fc(feature.view(img.shape[0],-1))
        return output

In [8]:
# test
net2=LeNet2()
X=torch.rand(1,1,28,28)

for blk in net.children():
    X=blk(X)
    print('out shape:',X.shape)

out shape: torch.Size([1, 16, 4, 4])
out shape: torch.Size([1, 10])


In [9]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

lr, num_epochs = 0.001, 1
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch05(net2, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

**小结**

模型训练时，批量归一化利用小批量的均值和标准差，不断调整神经网络的输出，从而使得神经网络的中间输出值更加稳定，

全连接层和卷积层的批量归一化各有不同，

批量归一化层和丢弃层一样，训练和验证模型下，结果不同，

使用nn.BatchNorm(num_features)简洁实现。